In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
adata1 = sc.read('/mnt/f/pvn/outer/new_protocal/step2_standard/step2_all_dedoubletcancer.h5ad')

In [ ]:
adata2 = adata1[adata1.obs['celltype_level1'].isin(['Monocytes'])]

In [ ]:
adata=adata2.raw.to_adata()

In [ ]:
sc.pp.highly_variable_genes(
    adata, n_top_genes=4000, flavor="seurat",batch_key="sample",
)

In [ ]:
adata.raw = adata.copy()

In [ ]:
highly_variable_genes = adata.var[adata.var['highly_variable']].index
hsp_genes = [gene for gene in highly_variable_genes if gene.startswith('Hsp')]
mt_genes = [gene for gene in highly_variable_genes if gene.startswith('mt-')]
rps_genes = [gene for gene in highly_variable_genes if gene.startswith('Rps') or gene.startswith('Rpl')]
print("Highly variable Hsp genes: ", hsp_genes)
print("Highly variable mt genes: ", mt_genes)
print("Highly variable rps genes: ", rps_genes)

In [ ]:
filtered_highly_variable_genes = [gene for gene in highly_variable_genes if gene not in hsp_genes and gene not in mt_genes and gene not in rps_genes]
adata.var['highly_variable'] = adata.var_names.isin(filtered_highly_variable_genes)
highly_variable_genes = adata.var['highly_variable']
print(f"Number of highly variable genes: {highly_variable_genes.sum()}")
adata = adata[:, adata.var["highly_variable"]]

In [ ]:
sc.pp.regress_out(adata, keys=["total_counts", "pct_counts_mt","pct_counts_hsp"])
sc.pp.scale(adata, max_value=10)
sc.pp.pca(adata, n_comps=100)
import scanpy.external as sce
sce.pp.harmony_integrate(adata, key="sample")
sc.pp.neighbors(adata, n_neighbors=30, n_pcs=100,use_rep='X_pca_harmony')
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.6,key_added='leiden_res0.6')

In [ ]:
sc.pl.umap(adata, color=['leiden_res0.6'], legend_loc="on data")

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden_res0.6', method='wilcoxon', use_raw=True) 

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).iloc[0:50,0:]

In [ ]:
annotations = {
    '0': 'Antigen TAMs', '1': 'Inflam TAMs', '2': 'Prolif TAMs',
    '3':'Lyve1 RTM TAMs',
    '4':'Angio TAMs','5':'IFN TAMs','6':'Monocytes','7':'Inflam TAMs',
}

In [ ]:
adata.obs['celltype_level2'] = adata.obs['leiden_res0.6'].map(annotations)
print(adata.obs[['leiden_res0.6', 'celltype_level2']])

In [ ]:
adata.write("/mnt/f/pvn/outer/new_protocal/step3_recluster/Macrophages.h5ad")

In [ ]:
adata.obs['celltype_level2'] = pd.Categorical(adata.obs['celltype_level2'])
celltype_order = [
    "Monocytes", "Inflam TAMs",  "Antigen TAMs","IFN TAMs",'Angio TAMs','Prolif TAMs','Lyve1 RTM TAMs',
]
adata.obs['celltype_level2'] = pd.Categorical(adata.obs['celltype_level2'], categories=celltype_order, ordered=True)

In [ ]:
Macrophage_genes=['Cd14','Vcan','Ly6c2',#mono'Ly6c1',
                  'Il1b', 'Ccl3', 'Cxcl1', 'Cxcl2', #inflam-TAM
                  'H2-Ea','H2-Eb1','H2-Ab1','Cx3cr1','Cd86','Cd74',#antigen
                  'Isg15','Ifit2','Ifit3', #IFN-TAM'Ifit1',
                  'Vegfa','Adam8','Bnip3','Mif', #Angio-TAM'Arg1','Slc2a1',
                  'Cdk1','Mki67','Stmn1','Top2a',#Prolif-TAM
                  'Lyve1','Folr2','Mrc1', #RTM-TAM
                  'Bin1','P2ry12'
                 ]

In [ ]:
sc.pl.dotplot(adata,Macrophage_genes, groupby='celltype_level2',
              standard_scale='var',
              colorbar_title='Normalized expression'
             )

In [ ]:
adr_genes=[ 'Adra1a','Adra1b','Adra2a','Adra2b','Adrb1','Adrb2','Adrb3' ]

In [ ]:
sc.pl.dotplot(adata,rtm_genes, groupby='celltype_level2',
              standard_scale='var',
             )

In [ ]:
gene = [
    'Tnfsf9', 'Ccl12', 'Ccl7', 'St3gal6', 'Col14a1', 'Marcksl1', 'Vcam1', 'Blnk', 'Bmp2', 'Ccr2', 
    'Egr1', 'Fnbp1l', 'Cbr2', 'Cyr61', 'F13a1', 'Fcrls', 'Gas7', 'Igfbp4', 'Lifr', 'Mgl2', 
    'Pmp22', 'Hivep2', 'Rbpj', 'Ccnd1', 'Il10', 'Itga6', 'Cd163', 'Cfh', 'Il21r', 'Rgs18', 
    'Apold1', 'S100a4', 'Akt3', 'Gpr165', 'Serpinf1', 'Ccl2', 'Ccl4', 'Ccl9', 'Cp', 'Il1rl1', 
    'Kitl', 'Clec10a', 'Lat2', 'Pf4', 'Tagap', 'Ch25h', 'Mef2c', 'Plxna4', 'Folr2', 'Gas6', 
    'Lyve1', 'Sdc4', 'Clec12a', 'Wwp1', 'Arap3', 'Ptgs2', 'Bend4', 'Gm8995', 'Nfkbiz', 'Socs3', 
    'Skil', 'Sesn1', '1700025G04Rik', 'Arhgap22', 'Cdk6', 'Cdr2', 'Dennd2c', 'Ptpro', 'Susd3', 
    'Atp8b4', 'Rtp4', 'Cxcl1', 'Cxcl2', 'Dlc1', 'Etv1', 'Ier3', 'Tln2', 'Tnfaip3', 'Zfp37', 
    'Abca9', 'Cd209a', 'Retnla', 'Mmp13', 'Olfml3', 'Man1a', 'Stard8', '1810011O10Rik', 'Cd40', 
    'Hpgd', 'Itgam', 'Tlr1'
]

In [ ]:
sc.tl.score_genes(adata,gene, ctrl_as_ref=True, ctrl_size=50, gene_pool=None, n_bins=25, 
                  score_name='NAM_score', random_state=0, copy=False, use_raw=True)

In [ ]:
import seaborn as sns

palette = sns.color_palette("Set2", n_colors=adata.obs['celltype_level2'].nunique())

sc.pl.violin(
    adata,
    keys=['NAM score'],
    groupby='celltype_level2',
    rotation=30,
    size=0,
    palette=palette,
    jitter=False,
    #show=False,          # 不显示在屏幕上
    save="macro_NAMscore_violin.pdf" 
)

In [ ]:
M1 = ["Irf1", "Cxcl11", "Cxcl9", "Marco", "Il1b", "Cd86", "Tnf", "Il2ra","Cxcl10", "Fcgr1a"]
M2 = ["Clec7a", "Gas7", "Ccl18", "Cd209", "Lipa", "F13a1", "Ctsd", "Ms4a4a", 
    "Maf", "Csf1r", "Ccl23", "Ccl7", "Hmox1", "Fn1", "Ccl2", "Ccl17", 
    "Il27ra", "Cxcr4", "Pparg"]

In [ ]:
sc.tl.score_genes(adata,M1, ctrl_as_ref=True, ctrl_size=50, gene_pool=None, n_bins=25, 
                  score_name='M1_celltype_level2', random_state=0, copy=False, use_raw=True)
sc.tl.score_genes(adata,M2, ctrl_as_ref=True, ctrl_size=50, gene_pool=None, n_bins=25, 
                  score_name='M2_celltype_level2', random_state=0, copy=False, use_raw=True)

In [ ]:
import seaborn as sns

# 使用 seaborn 调色板
palette = sns.color_palette("Set2", n_colors=adata.obs['celltype_level2'].nunique())

# 按 celltype_level2 画小提琴图
sc.pl.violin(
    adata,
    keys=['M1_celltype_level2'],
    groupby='celltype_level2',
    rotation=30,
    size=0,
    palette=palette,
    jitter=False,
    #show=False,          # 不显示在屏幕上
    save="macro_M1score_violin.pdf" 
)

In [ ]:
import seaborn as sns

# 使用 seaborn 调色板
palette = sns.color_palette("Set2", n_colors=adata.obs['celltype_level2'].nunique())

# 按 celltype_level2 画小提琴图
sc.pl.violin(
    adata,
    keys=['M2_celltype_level2'],
    groupby='celltype_level2',
    rotation=30,
    size=0,
    palette=palette,
    jitter=False,
    #show=False,          # 不显示在屏幕上
    save="macro_M2score_violin.pdf" 
)